# Data Exploration

In [48]:
import pandas
pandas.__version__

'2.1.1'

### Parse Files

In [67]:
import json

FILES = [
    'leagues.json',
    'player.json',
    'teams.json',
    'tournaments.json',
]

def parse(filename):
    f = open(filename)
    data = json.load(f)
    return data

leagues = parse('data/leagues.json')
players = parse('data/player.json')
teams = parse('data/teams.json')
tournaments = parse('data/tournaments.json')

print(leagues)

[{'id': '98767991299243165', 'name': 'LCS', 'slug': 'lcs', 'sport': 'lol', 'image': 'http://static.lolesports.com/leagues/LCSNew-01-FullonDark.png', 'lightImage': 'http://static.lolesports.com/leagues/LCSNew-01-FullonDark.png', 'darkImage': 'http://static.lolesports.com/leagues/LCSNew-01-FullonDark.png', 'region': 'NORTH AMERICA', 'priority': 1, 'displayPriority': {'position': 0, 'status': 'selected'}, 'tournaments': [{'id': '110303581083678395'}, {'id': '109517090066605615'}, {'id': '108206581962155974'}, {'id': '107458367237283414'}, {'id': '107458335260330212'}, {'id': '105658534671026792'}, {'id': '105788932118361426'}, {'id': '109428868589633757'}, {'id': '105522217230238828'}, {'id': '104174992692075107'}, {'id': '103462439438682788'}, {'id': '108300620375000370'}, {'id': '108294935083112239'}, {'id': '108288780705657082'}, {'id': '108283378783680768'}, {'id': '108007809203463350'}, {'id': '107784927885285215'}, {'id': '107744640257699883'}, {'id': '106972812538132507'}, {'id': '

In [69]:
from flatten_json import flatten

flatten(tournaments)

AssertionError: flatten requires a dictionary input

In [65]:
import pandas

t = pandas.DataFrame(tournaments)

df = pandas.json_normalize(tournaments, record_path=["stages"], meta=list(t.columns), meta_prefix="tournament.", record_prefix="stage.")
df

,stage.name,stage.type,stage.slug,stage.sections,tournament.id,tournament.leagueId,tournament.name,tournament.slug,tournament.sport,tournament.startDate,tournament.endDate,tournament.stages
0,Groups,None,groups,"[{'name': 'Group A', 'matches': [{'id': '11073...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,knockouts,None,knockouts,"[{'name': 'knockouts', 'matches': [{'id': '110...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
2,Regular Season,None,regular_season,"[{'name': 'Regular Season', 'matches': [{'id':...",110428723804419399,105266108767593290,Summer 2023,gll_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,Playoffs,None,playoffs,"[{'name': 'Playoffs', 'matches': [{'id': '1104...",110428723804419399,105266108767593290,Summer 2023,gll_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,Regular Season,None,regular_season,"[{'name': 'Regular Season', 'matches': [{'id':...",110424377524465827,107407335299756365,Summer 2023,elite_series_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
...,...,...,...,...,...,...,...,...,...,...,...,...
502,west,None,west,"[{'name': 'All-Star Exhibition', 'matches': [{...",105386292089445027,98767991295297326,ase_2020,ase_2020,lol,2020-12-18,2020-12-20,"[{'name': 'east', 'type': None, 'slug': 'east'..."
503,Regular Season,None,regular_season,"[{'name': 'Regular Season', 'matches': [{'id':...",105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."
504,Playoffs,None,playoffs,"[{'name': 'Playoffs', 'matches': [{'id': '1058...",105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."
505,Regular Season,None,regular_season,"[{'name': 'Round Robin', 'matches': [{'id': '1...",104237136838348356,101382741235120470,lla-closing-2020,lla-closing-2020,lol,2020-06-20,2020-09-06,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [51]:
t

,id,leagueId,name,slug,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,Summer 2023,gll_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,Summer 2023,elite_series_summer_2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,Summer 2023,superliga_summer_2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,Summer 2023,ultraliga_summer_2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."
...,...,...,...,...,...,...,...,...
243,103462439438682788,98767991299243165,LCS Spring 2020,lcs_spring_2020,lol,2020-01-24,2020-04-20,"[{'name': 'Regular Season', 'type': None, 'slu..."
244,104253013242629769,101097443346691685,TAL Summer 2020,tal_summer_2020,lol,2020-06-08,2020-08-10,"[{'name': 'Regular Season', 'type': None, 'slu..."
245,105386292089445027,98767991295297326,ase_2020,ase_2020,lol,2020-12-18,2020-12-20,"[{'name': 'east', 'type': None, 'slug': 'east'..."
246,105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [58]:
pandas.json_normalize(df.to_json())

NotImplementedError: 

In [ ]:
df.explode('stage.sections')

,stage.name,stage.type,stage.slug,stage.sections,tournament.id,tournament.leagueId,tournament.name,tournament.slug,tournament.sport,tournament.startDate,tournament.endDate,tournament.stages
0,Groups,None,groups,"{'name': 'Group A', 'matches': [{'id': '110733...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
0,Groups,None,groups,"{'name': 'Group B', 'matches': [{'id': '110733...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
0,Groups,None,groups,"{'name': 'Group C', 'matches': [{'id': '110733...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
0,Groups,None,groups,"{'name': 'Group D', 'matches': [{'id': '110733...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,knockouts,None,knockouts,"{'name': 'knockouts', 'matches': [{'id': '1107...",110733838935136200,109518549825754242,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
...,...,...,...,...,...,...,...,...,...,...,...,...
503,Regular Season,None,regular_season,"{'name': 'Regular Season', 'matches': [{'id': ...",105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."
504,Playoffs,None,playoffs,"{'name': 'Playoffs', 'matches': [{'id': '10581...",105562379823435874,105266118689416013,baltic_masters_2021_spring,baltic_masters_2021_spring,lol,2021-01-16,2021-03-12,"[{'name': 'Regular Season', 'type': None, 'slu..."
505,Regular Season,None,regular_season,"{'name': 'Round Robin', 'matches': [{'id': '10...",104237136838348356,101382741235120470,lla-closing-2020,lla-closing-2020,lol,2020-06-20,2020-09-06,"[{'name': 'Regular Season', 'type': None, 'slu..."
505,Regular Season,None,regular_season,"{'name': 'Round Robin', 'matches': [{'id': '10...",104237136838348356,101382741235120470,lla-closing-2020,lla-closing-2020,lol,2020-06-20,2020-09-06,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [ ]:
pandas.json_normalize(df["stage.sections"])

,0,1,2,3
0,"{'name': 'Group A', 'matches': [{'id': '110733...","{'name': 'Group B', 'matches': [{'id': '110733...","{'name': 'Group C', 'matches': [{'id': '110733...","{'name': 'Group D', 'matches': [{'id': '110733..."
1,"{'name': 'knockouts', 'matches': [{'id': '1107...",None,None,None
2,"{'name': 'Regular Season', 'matches': [{'id': ...",None,None,None
3,"{'name': 'Playoffs', 'matches': [{'id': '11042...",None,None,None
4,"{'name': 'Regular Season', 'matches': [{'id': ...",None,None,None
...,...,...,...,...
502,"{'name': 'All-Star Exhibition', 'matches': [{'...",None,None,None
503,"{'name': 'Regular Season', 'matches': [{'id': ...",None,None,None
504,"{'name': 'Playoffs', 'matches': [{'id': '10581...",None,None,None
505,"{'name': 'Round Robin', 'matches': [{'id': '10...","{'name': 'Round Robin', 'matches': [{'id': '10...",None,None
